<h1>Stations data table</h1>

<h3>IMPORTS</h3>

In [ ]:
#IMPORTS
import pandas as pd
import sqlite3
import os

<h3>METHODS</h3>

In [ ]:
#process 1 csv file 

def extract_data(file_path):

    try:
        df = pd.read_csv(file_path, encoding='latin1', header = None, keep_default_na=False, index_col=None)
    except Exception as e:
        raise Exception("ERROR READING CSV:", file_path, "-- WITH ERROR:", e)   
    
    df = df.drop([0,1,2,3,4,5,6])  # Drop the first 7 rows


    try:
        df = df.iloc[:, [0, 1, 3, 5, 6, 7, 8 , 9 ]]   
    except:
        raise Exception("column names not properly formatted in file")

    row1 = df.iloc[0]
    row2 = df.iloc[1]
    row3 = df.iloc[2]
    row4 = df.iloc[3]
    df = df.drop(df.index[-1])


    def combine_rows(row1, row2):
        combined_row = {}
        for col in row1.index:
            combined_row[col] = (str(row1[col]) + " " + str(row2[col])).strip()
        return pd.DataFrame([combined_row]).iloc[0]  
        

    header_row = combine_rows(combine_rows(combine_rows(row1, row2), row3), row4)

    #Set top row as header row values
    df.columns = header_row.tolist()
    df = df.drop(df.index[[0,1,2,3]])  # Drop the first four rows which are now redundant

    df = df.reset_index(drop=True)  # Reset the index

    return df



In [ ]:
def insert_data_from_file(file_path, connection, db_table):
    try:
        df = extract_data(file_path)
    except Exception as e:
        return e

    try:
        df.to_sql(db_table, connection, if_exists='append', index=False)   
    except Exception as e:
        return e

<h3>Establish file paths & connections</h3>

In [ ]:
# File paths
data_folder_path = "./../Data/"
db_file_path = "./../WeatherStationDB.db"

#connection to database
db_conn = sqlite3.connect(db_file_path)
db_table = 'weather_data'


<h3>RESET TABLE ROWS</h3>

In [ ]:
cursor = db_conn.cursor()

# SQL statement to delete all rows from the table
sql_delete_all_rows = f"DELETE FROM {db_table};"

try:
    # Execute the DELETE statement
    cursor.execute(sql_delete_all_rows)

    # Commit the changes to the database
    db_conn.commit()

    print(f"All rows deleted from table '{db_table}'.")

except sqlite3.Error as e:
    print(f"Error deleting rows: {e}")

finally:
    # Close the cursor
    cursor.close()


All rows deleted from table 'weather_data'.


<h3>MAKE UNIQUE TABLE DATA</h3>

In [ ]:
# cursor = db_conn.cursor()


# sql_create_unique_table = "CREATE TABLE weather_data_temp AS SELECT DISTINCT * FROM weather_data"
# sql_drop_old_table = "DROP TABLE weather_data"
# sql_create_new_table = "CREATE TABLE weather_data AS SELECT DISTINCT * FROM weather_data_temp"
# sql_drop_temp_table = "DROP TABLE weather_data_temp"

# try:
#     # Execute the DELETE statement
#     cursor.execute(sql_create_unique_table)
#     cursor.execute(sql_drop_old_table)
#     cursor.execute(sql_create_new_table)
#     cursor.execute(sql_drop_temp_table)


#     # Commit the changes to the database
#     db_conn.commit()

#     print(f"Distinct '{db_table}' table has been created.")

# except sqlite3.Error as e:
#     print(f"Error creating distinct table: {e}")

# finally:
#     # Close the cursor
#     cursor.close()

Distinct 'weather_data' table has been created.


<h3>INSERT ALL Data into weather_data table</h3>

In [ ]:
#extract data from files in folder and insert into db

states = ['vic', 'wa','tas','act','nt']


for state in states:
    # Get the list of all files in directory
    try:
        state_data_path = data_folder_path + state
        dir_list = os.listdir(state_data_path)
    except Exception as e:
        print(e)
        continue

    for data_file in dir_list:
        # get relative data file path
        data_file_path = state_data_path + "/" + data_file

        try:
            df = extract_data(data_file_path)
        except Exception as e:
            continue

        # 'if_exists' can be 'fail', 'replace', or 'append'
        # 'index=False' prevents writing the DataFrame index as a column in the database
        try:
            df.to_sql(db_table, db_conn, if_exists='append', index=False)
        except Exception as e:
            print("error with file:", data_file_path, "ERROR CAUGHT:", e)
            pass

[WinError 3] The system cannot find the path specified: './Data/wa'
